In [ ]:
# encoding: UTF-8
# original source : https://github.com/GoogleCloudPlatform/tensorflow-without-a-phd/tree/master/tensorflow-mnist-tutorial
# 2018.12 : modified by Seungkwon Lee(kahnlee@naver.com)

import tensorflow as tf
import tensorflowvisu
import mnistdata
print("Tensorflow version " + tf.__version__)
import matplotlib.pyplot as plt
import numpy as np

tf.set_random_seed(0)

# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   # 784 = 28*28
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1 [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2 [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3 [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4 [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y5 [batch, 10]

# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = mnistdata.read_data_sets("data", one_hot=True, reshape=False)

saver = tf.train.Saver(max_to_keep = 3)

# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 10])

#neuron 개수
# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 100
N = 60
O = 30
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  # 784 = 28 * 28
B1 = tf.Variable(tf.zeros([L]))
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.zeros([M]))
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.zeros([N]))
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.zeros([O]))
W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

# The model
XX = tf.reshape(X, [-1, 784])
Y1 = tf.nn.sigmoid(tf.matmul(XX, W1) + B1)
Y2 = tf.nn.sigmoid(tf.matmul(Y1, W2) + B2)
Y3 = tf.nn.sigmoid(tf.matmul(Y2, W3) + B3)
Y4 = tf.nn.sigmoid(tf.matmul(Y3, W4) + B4)
Ylogits = tf.nn.sigmoid(tf.matmul(Y4, W5) + B5)
Y = tf.nn.softmax(Ylogits)

# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# training step, learning rate = 0.003
learning_rate = 0.003
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


train_acc_list = []
test_acc_list = []
train_loss_list = []
test_loss_list = []
# run
for i in range(10000 + 1) :

    batch_X, batch_Y = mnist.train.next_batch(100)
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X : batch_X, Y_ : batch_Y})
    print("training : ", i, ' accuracy = ', '{:7.4f}'.format(a), ' loss = ', c)
    train_acc_list.append(a)
    train_loss_list.append(c)

    # test_batch_X, test_batch_Y = mnist.test.next_batch(100)  ==> never use mini batch!!
    # sess.run(train_step, feed_dict={X: test_batch_X, Y_: test_batch_Y})  ==> never run train_step on test data!!
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
    print("testing  : ",i, ' accuracy = ', '{:7.4f}'.format(a), ' loss = ', c)
    test_acc_list.append(a)
    test_loss_list.append(c)

    sess.run(train_step, feed_dict={X : batch_X, Y_ : batch_Y} )
    
    if(i%500 ==0 and i!=0):
        saver.save(sess, '../model/mnist_HL5_sigmoid')

# draw graph : accuracy
x = np.arange(len(train_acc_list))
plt.figure(1) 
plt.plot(x, train_acc_list,  label='train', markevery=1)
plt.plot(x, test_acc_list, label='test', markevery=1)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
# plt.show()

# draw graph : loss
x = np.arange(len(train_loss_list))
plt.figure(2) 
plt.plot(x, train_loss_list,  label='train', markevery=1)
plt.plot(x, test_loss_list, label='test', markevery=1)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.ylim(0, 100)
plt.legend(loc='upper right')
plt.show()

Tensorflow version 1.8.0
training :  0  accuracy =   0.0600  loss =  231.403
testing  :  0  accuracy =   0.0980  loss =  230.531
training :  1  accuracy =   0.0700  loss =  229.642
testing  :  1  accuracy =   0.0958  loss =  230.449
training :  2  accuracy =   0.1200  loss =  229.27
testing  :  2  accuracy =   0.0980  loss =  230.407
training :  3  accuracy =   0.1400  loss =  230.795
testing  :  3  accuracy =   0.1028  loss =  230.408
training :  4  accuracy =   0.0500  loss =  231.64
testing  :  4  accuracy =   0.1028  loss =  230.405
training :  5  accuracy =   0.0400  loss =  232.221
testing  :  5  accuracy =   0.1028  loss =  230.389
training :  6  accuracy =   0.0700  loss =  231.317
testing  :  6  accuracy =   0.1028  loss =  230.348
training :  7  accuracy =   0.1600  loss =  229.333
testing  :  7  accuracy =   0.1028  loss =  230.295
training :  8  accuracy =   0.0900  loss =  230.846
testing  :  8  accuracy =   0.1028  loss =  230.252
training :  9  accuracy =   0.0800  loss 

testing  :  77  accuracy =   0.4690  loss =  204.191
training :  78  accuracy =   0.4200  loss =  203.605
testing  :  78  accuracy =   0.4693  loss =  203.667
training :  79  accuracy =   0.4700  loss =  201.21
testing  :  79  accuracy =   0.4708  loss =  203.116
training :  80  accuracy =   0.4000  loss =  203.58
testing  :  80  accuracy =   0.4712  loss =  202.622
training :  81  accuracy =   0.4400  loss =  199.897
testing  :  81  accuracy =   0.4695  loss =  202.22
training :  82  accuracy =   0.4400  loss =  202.03
testing  :  82  accuracy =   0.4703  loss =  201.85
training :  83  accuracy =   0.4500  loss =  200.132
testing  :  83  accuracy =   0.4729  loss =  201.343
training :  84  accuracy =   0.5400  loss =  197.929
testing  :  84  accuracy =   0.4781  loss =  200.838
training :  85  accuracy =   0.4800  loss =  201.505
testing  :  85  accuracy =   0.4780  loss =  200.376
training :  86  accuracy =   0.5200  loss =  199.991
testing  :  86  accuracy =   0.4766  loss =  200.00

testing  :  155  accuracy =   0.4672  loss =  184.409
training :  156  accuracy =   0.4700  loss =  182.098
testing  :  156  accuracy =   0.4654  loss =  184.273
training :  157  accuracy =   0.5100  loss =  182.857
testing  :  157  accuracy =   0.4607  loss =  184.2
training :  158  accuracy =   0.4900  loss =  180.812
testing  :  158  accuracy =   0.4536  loss =  184.222
training :  159  accuracy =   0.5000  loss =  185.956
testing  :  159  accuracy =   0.4482  loss =  184.303
training :  160  accuracy =   0.5000  loss =  183.368
testing  :  160  accuracy =   0.4394  loss =  184.601
training :  161  accuracy =   0.4800  loss =  180.701
testing  :  161  accuracy =   0.4346  loss =  184.696
training :  162  accuracy =   0.4600  loss =  182.496
testing  :  162  accuracy =   0.4319  loss =  184.573
training :  163  accuracy =   0.3700  loss =  187.634
testing  :  163  accuracy =   0.4329  loss =  184.155
training :  164  accuracy =   0.4300  loss =  184.997
testing  :  164  accuracy =   

testing  :  233  accuracy =   0.4592  loss =  178.579
training :  234  accuracy =   0.5200  loss =  176.265
testing  :  234  accuracy =   0.4660  loss =  178.285
training :  235  accuracy =   0.5300  loss =  178.102
testing  :  235  accuracy =   0.4701  loss =  178.053
training :  236  accuracy =   0.4400  loss =  177.883
testing  :  236  accuracy =   0.4705  loss =  177.965
training :  237  accuracy =   0.4000  loss =  183.213
testing  :  237  accuracy =   0.4742  loss =  177.957
training :  238  accuracy =   0.4600  loss =  179.572
testing  :  238  accuracy =   0.4991  loss =  177.885
training :  239  accuracy =   0.5500  loss =  173.991
testing  :  239  accuracy =   0.4605  loss =  177.844
training :  240  accuracy =   0.4000  loss =  178.044
testing  :  240  accuracy =   0.4547  loss =  177.874
training :  241  accuracy =   0.4900  loss =  174.211
testing  :  241  accuracy =   0.4552  loss =  177.998
training :  242  accuracy =   0.4500  loss =  174.329
testing  :  242  accuracy = 

testing  :  309  accuracy =   0.4981  loss =  175.142
training :  310  accuracy =   0.5500  loss =  172.713
testing  :  310  accuracy =   0.4974  loss =  175.133
training :  311  accuracy =   0.5200  loss =  176.587
testing  :  311  accuracy =   0.4986  loss =  175.202
training :  312  accuracy =   0.5400  loss =  173.464
testing  :  312  accuracy =   0.4964  loss =  175.181
training :  313  accuracy =   0.5100  loss =  173.972
testing  :  313  accuracy =   0.4930  loss =  174.937
training :  314  accuracy =   0.5000  loss =  173.979
testing  :  314  accuracy =   0.4871  loss =  174.732
training :  315  accuracy =   0.4800  loss =  172.03
testing  :  315  accuracy =   0.4691  loss =  174.551
training :  316  accuracy =   0.4400  loss =  174.532
testing  :  316  accuracy =   0.4685  loss =  174.498
training :  317  accuracy =   0.3600  loss =  178.036
testing  :  317  accuracy =   0.4709  loss =  174.417
training :  318  accuracy =   0.5100  loss =  174.307
testing  :  318  accuracy =  